In [12]:
from codebase.classes_ibis_lvm import ParticlesLVM
from codebase.classes_data import Data
from codebase.ibis import model_phonebook, essl
import pandas as pd
import numpy as np
from codebase.file_utils import (
    save_obj,
    load_obj,
    make_folder,
    path_backslash
)
from codebase.ibis import essl, exp_and_normalise, run_stan_model
from run_ibis import run_ibis

from tqdm import tqdm
from scipy.special import logsumexp
import altair as alt
from codebase.plot import plot_density, get_post_df, plot_correlations
alt.data_transformers.disable_max_rows()


DataTransformerRegistry.enable('default')

## 2 factor Sim EZ

In [2]:
existing_directory = None
task_handle = 'ez_lvm'
gen_model = 0

if existing_directory is None:
    log_dir = make_folder(task_handle)  
    print("\n\nCreating new directory: %s" % log_dir)

else:
    log_dir = existing_directory
    log_dir = path_backslash(log_dir)
    print("\n\nReading from existing directory: %s" % log_dir)




Creating new directory: ./log/20201228_162426_ez_lvm/


## Create Data

In [7]:
# generate data
exp_data = Data(
    name = task_handle, 
    model_num = 6, 
    size = 50,
    random_seed = 6
    )
    
exp_data.generate()

In [4]:
# particles = ParticlesLVM(
#     name = 'ibis_lvm',
#     model_num = 8,
#     size = 100,
#     bundle_size=50,
#     param_names = ['param_names'],
#     latent_names = ['latent_names'],
#     latent_model_num= 1
# )
# particles.set_log_dir(log_dir)
# particles.compile_model()


INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_227ba6f1091299bc7d305004d9fd26c2 NOW.


## Run HMC

In [9]:
sm = load_obj('sm', 'log/compiled_models/model8/')

fit_run = run_stan_model(
    data = exp_data.get_stan_data(),
    compiled_model = sm,
    num_samples = 1000, 
    num_warmup = 1000,
    num_chains = 1,
    adapt_engaged=True
    )

In [10]:
param_names = ['beta', 'alpha']

ps = fit_run.extract(
        permuted=False, pars=param_names)

save_obj(ps, 'ps_hmc', log_dir)

## Run IBIS

In [6]:
# load existing results

# log_dir = 'log/20201218_172340_ez_comp/'
# particles = load_obj('particles', log_dir)

In [13]:
gen_model = False
model_num = 7
degeneracy_limit = 0.5


param_names = model_phonebook(model_num)['param_names']
latent_names = model_phonebook(model_num)['latent_names']
jitter_corrs = dict()
for p in param_names:
    jitter_corrs[p] = np.zeros(exp_data.size)
particles = ParticlesLVM(
    name = 'ibis_lvm',
    model_num = model_num,
    size = 100,
    bundle_size=50,
    param_names = param_names,
    latent_names = latent_names,
    latent_model_num= 1
)
particles.set_log_dir(log_dir)
if gen_model:
    particles.compile_prior_model()
    particles.compile_model()
else:
    particles.load_prior_model()
    particles.load_model()

log_lklhds = np.empty(exp_data.size)
degeneracy_limit = 0.5


particles.sample_prior_particles(exp_data.get_stan_data()) # sample prior particles
particles.reset_weights() # set weights to 0


for t in tqdm(range(exp_data.size)):    
    particles.sample_latent_variables(exp_data.get_stan_data_at_t(t))
    particles.get_bundle_weights(exp_data.get_stan_data_at_t(t))

    particles.incremental_weights = np.mean(np.squeeze(particles.latent_weights), axis=1)
    log_lklhds[t] =  particles.get_loglikelihood_estimate()
    

    if (essl(particles.weights) < degeneracy_limit * particles.size) and (t+1) < exp_data.size:
        
        particles.resample_particles()    
        
        particles.sample_latent_variables(exp_data.get_stan_data_upto_t(t+1))
        particles.get_bundle_weights(exp_data.get_stan_data_upto_t(t+1))

        particles.sample_latent_particles_star(exp_data.get_stan_data_upto_t(t+1))
        particles.sample_latent_var_given_theta(exp_data.get_stan_data_upto_t(t+1))

        
        ## add corr of param before jitter
        pre_jitter = dict()
        for p in param_names:
            pre_jitter[p] = particles.particles[p].flatten()
        ####

        particles.jitter(exp_data.get_stan_data_upto_t(t+1))

        ## add corr of param
        for p in param_names:
            jitter_corrs[p][t] = np.corrcoef(pre_jitter[p],particles.particles[p].flatten())[0,1]          
        ####

        particles.reset_weights()
    else:
        particles.update_weights()


100%|██████████| 50/50 [04:07<00:00,  4.94s/it]


In [14]:
save_obj(particles, 'particles', log_dir)
save_obj(t, 't', log_dir)
save_obj(log_lklhds, 'log_lklhds', log_dir)

## Plot Both results

In [15]:
# have to resample particles to get rid of weights
# particles = ibis['particles']
particles.resample_particles()

In [17]:
param = 'beta'
df = get_post_df(ps[param]) 
df['source'] = 'hmc'
df2 = get_post_df(particles.particles[param])
df2['source'] = 'ibis'
plot_density(pd.concat([df,df2])) 
# &\
# plot_correlations(ibis['jitter_corrs'][param], height = 150)

INFO:numexpr.utils:Note: NumExpr detected 24 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


alt.Chart(...)

In [10]:
param = 'alpha'
df = get_post_df(ps[param]) 
df['source'] = 'hmc'
df2 = get_post_df(particles.particles[param])
df2['source'] = 'ibis'
plot_density(pd.concat([df,df2])) &\
plot_correlations(ibis['jitter_corrs'][param], height = 150)

alt.VConcatChart(...)